In [ ]:
############  add brightness on part of image #################################

In [ ]:
from PIL import Image
import math
import numpy as np
def spotlight(img: Image, center: (int, int), radius: int) -> Image:
    width, height = img.size
    overlay_color = (0, 0, 0, 40)
    img_overlay = Image.new(size=img.size, color=overlay_color, mode='RGBA')
    x1 =  np.random.randint(50,500)
    x2 =  np.random.randint(501,1600)
    y1 =  np.random.randint(50,300)
    y2 =  np.random.randint(301,1200)
    x11 =  np.random.randint(50,500)
    x21 =  np.random.randint(501,1600)
    y11 =  np.random.randint(50,300)
    y21 =  np.random.randint(301,1200)
    for x in range(x1,x2):
        for y in range(y1,y2):
            dx = x - center[0]
            dy = y - center[1]
            distance = math.sqrt(dx * dx + dy * dy)
            img_overlay.putpixel((x, y), (0, 0, 0, 0))
    for x in range(x11,x21):
        for y in range(y11,y21):
            dx = x - center[0]
            dy = y - center[1]
            distance = math.sqrt(dx * dx + dy * dy)
#            img_overlay.putpixel((x, y), (0, 0, 0, 0))
            if distance < radius:
                 img_overlay.putpixel((x, y), (0, 0, 0, 0))
                    
    img.paste(img_overlay, None, mask=img_overlay)
    return img

In [13]:
orig_file_name = '51'
img = Image.open('{}.png'.format(orig_file_name))
spotlight_img = spotlight(img, (500, 700), 100)
spotlight_img.save('spotlight_{}.jpg'.format(orig_file_name))

In [ ]:
imagelist = [file for file in os.listdir("Art_Dataset/test_img/") if file.endswith('.png')]
print(len(imagelist))
if not os.path.exists("Art_Dataset/spotlight"):
    os.mkdir("Art_Dataset/spotlight/")

In [ ]:
for image in imagelist:
    img = Image.open("Art_Dataset/test_img/{}".format(image))
    img = spotlight(img, (500, 700), 100)
    img.save("Art_Dataset/spotlight/{}".format(image))

In [9]:
#https://stackoverflow.com/questions/33603304/create-a-spotlight-in-an-image-using-python

In [ ]:
######## sp noise on range of image#################################

In [15]:
import numpy as np
import random
import cv2

def sp_noise(image,prob):
    '''
    Add salt and pepper noise to image
    prob: Probability of the noise
    '''
    output = np.zeros((400,600),np.uint8)
    thres = 1 - prob 
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            rdn = random.random()
            if rdn < prob:
                output[i][j] = 0
            elif rdn > thres:
                output[i][j] = 255
            else:
                output[i][j] = image[i][j]
    return output

image = cv2.imread('51.png',0) # Only for grayscale image
noise_img = sp_noise(image,0.05)
cv2.imwrite('sp_noise.jpg', noise_img)

IndexError: index 600 is out of bounds for axis 0 with size 600

In [ ]:
##########  ramdom controlling bluring and lighting #################

In [16]:
def _decayed_value_in_norm(x, max_value, min_value, center, range):
    """
    decay from max value to min value following Gaussian/Normal distribution
    """
    radius = range / 3
    center_prob = norm.pdf(center, center, radius)
    x_prob = norm.pdf(x, center, radius)
    x_value = (x_prob / center_prob) * (max_value - min_value) + min_value
    return x_value
def _decayed_value_in_linear(x, max_value, padding_center, decay_rate):
    """
    decay from max value to min value with static linear decay rate.
    """
    x_value = max_value - abs(padding_center - x) * decay_rate
    if x_value < 0:
        x_value = 1
    return x_value

In [20]:
import cv2
import numpy as np
import random
from scipy.stats import norm

def generate_parallel_light_mask(mask_size,
                                 position=None,
                                 direction=None,
                                 max_brightness=255,
                                 min_brightness=0,
                                 mode="gaussian",
                                 linear_decay_rate=None):
    if position is None:
        pos_x = random.randint(0, mask_size[0])
        pos_y = random.randint(0, mask_size[1])
    else:
        pos_x = position[0]
        pos_y = position[1]
    if direction is None:
        direction = random.randint(0, 360)
        print("Rotate degree: ", direction)
    if linear_decay_rate is None:
        if mode == "linear_static":
            linear_decay_rate = random.uniform(0.2, 2)
        if mode == "linear_dynamic":
            linear_decay_rate = (max_brightness - min_brightness) / max(mask_size)
    assert mode in ["linear_dynamic", "linear_static", "gaussian"], \
        "mode must be linear_dynamic, linear_static or gaussian"
    padding = int(max(mask_size) * np.sqrt(2))
    # add padding to satisfy cropping after rotating
    canvas_x = padding * 2 + mask_size[0]
    canvas_y = padding * 2 + mask_size[1]
    mask = np.zeros(shape=(canvas_y, canvas_x), dtype=np.float32)
    # initial mask's up left corner and bottom right corner coordinate
    init_mask_ul = (int(padding), int(padding))
    init_mask_br = (int(padding+mask_size[0]), int(padding+mask_size[1]))
    init_light_pos = (padding + pos_x, padding + pos_y)
    # fill in mask row by row with value decayed from center
    for i in range(canvas_y):
        if mode == "linear_static":
            i_value = _decayed_value_in_linear(i, max_brightness, init_light_pos[1], linear_decay_rate)
        elif mode == "gaussian":
            i_value = _decayed_value_in_norm(i, max_brightness, min_brightness, init_light_pos[1], mask_size[1])
        else:
            i_value = 0
        mask[i] = i_value
    # rotate mask
    rotate_M = cv2.getRotationMatrix2D(init_light_pos, direction, 1)
    mask = cv2.warpAffine(mask, rotate_M, (canvas_x,  canvas_y))
    # crop
    mask = mask[init_mask_ul[1]:init_mask_br[1], init_mask_ul[0]:init_mask_br[0]]
    mask = np.asarray(mask, dtype=np.uint8)
    # add median blur
    mask = cv2.medianBlur(mask, 9)
    mask = 255 - mask
    return mask

In [21]:
def add_parallel_light(image, light_position=None, direction=None, max_brightness=255, min_brightness=0,
                       mode="linear_static", linear_decay_rate=None, transparency=None):
    """
    Add mask generated from parallel light to given image
    """
    if transparency is None:
        transparency = random.uniform(0.5, 0.85)
    frame = cv2.imread(image)
    height, width, _ = frame.shape
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    mask = generate_parallel_light_mask(mask_size=(width, height),
                                        position=light_position,
                                        direction=direction,
                                        max_brightness=max_brightness,
                                        min_brightness=min_brightness,
                                        mode=mode,
                                        linear_decay_rate=linear_decay_rate)
    hsv[:, :, 2] = hsv[:, :, 2] * transparency + mask * (1 - transparency)
    frame = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    frame[frame > 255] = 255
    frame = np.asarray(frame, dtype=np.uint8)
    return frame

In [22]:
img = "51.png"
img =add_parallel_light(img)
cv2.imwrite('light.jpg', img)

Rotate degree:  280


True

In [ ]:
imagelist = [file for file in os.listdir("Art_Dataset/test_img/") if file.endswith('.png')]
print(len(imagelist))
if not os.path.exists("Art_Dataset/add_parallel_light"):
    os.mkdir("Art_Dataset/add_parallel_light/")

In [ ]:
for image in imagelist:
    img = Image.open("Art_Dataset/test_img/{}".format(image))
    img = add_parallel_light(img)
    img.save("Art_Dataset/add_parallel_light/{}".format(image))

In [104]:
#create rainy snow shadow data for autonomous driving####################
#https://www.freecodecamp.org/news/image-augmentation-make-it-rain-make-it-snow-how-to-modify-a-photo-with-machine-learning-163c0cb3843f/

In [56]:
def add_brightness(image): 
    image_HLS = cv2.cvtColor(image,cv2.COLOR_RGB2HLS) 
    ## Conversion to HLS    
    image_HLS = np.array(image_HLS, dtype = np.float64)   
    random_brightness_coefficient = np.random.uniform()+0.5
    ## generates value between 0.5 and 1.5    
    image_HLS[:,:,1] = image_HLS[:,:,1]*random_brightness_coefficient
    ## scale pixel values up or down for channel 1(Lightness)    
    image_HLS[:,:,1][image_HLS[:,:,1]>255]  = 255
    ##Sets all values above 255 to 255    
    image_HLS = np.array(image_HLS, dtype = np.uint8)    
    image_RGB = cv2.cvtColor(image_HLS,cv2.COLOR_HLS2RGB) 
    ## Conversion to RGB    
    return image_RGB

In [57]:
image = cv2.imread('51.png')
img = add_brightness(image)
cv2.imwrite('sunny.jpg', img)

True

In [58]:
def generate_shadow_coordinates(imshape, no_of_shadows=1): 
    vertices_list=[]    
    for index in range(no_of_shadows):
        vertex=[] 
        for dimensions in range(np.random.randint(3,15)):
            ## Dimensionality of the shadow polygon 
            vertex.append(( imshape[1]*np.random.uniform(),imshape[0]//3+imshape[0]*np.random.uniform()))     
            vertices = np.array([vertex], dtype=np.int32) ## single shadow vertices        
            vertices_list.append(vertices)   
            return vertices_list ## List of shadow vertices

In [59]:
def add_shadow(image,no_of_shadows=1):
    image_HLS = cv2.cvtColor(image,cv2.COLOR_RGB2HLS) ## Conversion to HLS  
    mask = np.zeros_like(image)    
    imshape = image.shape   
    vertices_list= generate_shadow_coordinates(imshape, no_of_shadows) #3 getting list of shadow vertices  
    for vertices in vertices_list:   
        cv2.fillPoly(mask, vertices, 255) ## adding all shadow polygons on empty mask, single 255 denotes only red channel  
        image_HLS[:,:,1][mask[:,:,0]==255] = image_HLS[:,:,1][mask[:,:,0]==255]*0.5   ## if red channel is hot, image's "Lightness" channel's brightness is lowered   
        image_RGB = cv2.cvtColor(image_HLS,cv2.COLOR_HLS2RGB) ## Conversion to RGB   
        return image_RGB

In [61]:
image = cv2.imread('51.png')
img = add_shadow(image)
cv2.imwrite('shadows.jpg', img)

True

In [62]:
def add_snow(image):
    image_HLS = cv2.cvtColor(image,cv2.COLOR_RGB2HLS) ## Conversion to HLS  
    image_HLS = np.array(image_HLS, dtype = np.float64)  
    brightness_coefficient = 2.5   
    snow_point=140 ## increase this for more snow   
    image_HLS[:,:,1][image_HLS[:,:,1]<snow_point] = image_HLS[:,:,1][image_HLS[:,:,1]<snow_point]*brightness_coefficient ## scale pixel values up for channel 1(Lightness) 
    image_HLS[:,:,1][image_HLS[:,:,1]>255]  = 255 ##Sets all values above 255 to 255  
    image_HLS = np.array(image_HLS, dtype = np.uint8)   
    image_RGB = cv2.cvtColor(image_HLS,cv2.COLOR_HLS2RGB) ## Conversion to RGB    
    return image_RGB

In [63]:
image = cv2.imread('51.png')
img = add_shadow(image)
cv2.imwrite('snow.jpg', img)

True

In [131]:
def generate_random_lines(imshape,slant,drop_length):
    drops=[]    
    for i in range(1400): ## If You want heavy rain, try increasing this       
        if slant<0:           
            x= np.random.randint(slant,imshape[1])      
        else:   
            x= np.random.randint(0,imshape[1]-slant)    
            y= np.random.randint(0,imshape[0]-drop_length)      
            drops.append((x,y))  
    return drops           
def add_rain(image):      
    imshape = image.shape  
    slant_extreme=7    
    slant= np.random.randint(-slant_extreme,slant_extreme)   
    drop_length=10  
    drop_width=2   
    drop_color=(210,210,210) ## a shade of gray   
    rain_drops= generate_random_lines(imshape,slant,drop_length)      
    for rain_drop in rain_drops:       
        cv2.line(image,(rain_drop[0],rain_drop[1]),(rain_drop[0]+slant,rain_drop[1]+drop_length),drop_color,drop_width)   
    image= cv2.blur(image,(4,4)) ## rainy view are blurry    
    brightness_coefficient = 0.9 ## rainy days are usually shady   
    image_HLS = cv2.cvtColor(image,cv2.COLOR_RGB2HLS) ## Conversion to HLS 
    image_HLS[:,:,1] = image_HLS[:,:,1]*brightness_coefficient ## scale pixel values down for channel 1(Lightness) 
    image_RGB = cv2.cvtColor(image_HLS,cv2.COLOR_HLS2RGB) ## Conversion to RGB   
    return image_RGB

In [132]:
image = cv2.imread('51.png')
img = add_rain(image)
cv2.imwrite('rain.jpg', img)

True

In [ ]:
##############  end ##################

In [ ]:
############3  adjusting lighting gamma change ###################3

In [27]:
import cv2
import numpy as np

def adjust_gamma(image, gamma=1.0):
    invGamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** invGamma) * 255 for i in np.arange(0, 256)]).astype("uint8")
    return cv2.LUT(image, table)

x = '51.png'  #location of the image
original = cv2.imread(x, 1)
gamma = 1.5                                   # change the value here to get different result
adjusted = adjust_gamma(original, gamma=gamma)
cv2.imwrite('gamma.jpg', adjusted)

True

In [ ]:
imagelist = [file for file in os.listdir("Art_Dataset/test_img/") if file.endswith('.png')]
print(len(imagelist))
if not os.path.exists("Art_Dataset/adjust_gamma"):
    os.mkdir("Art_Dataset/adjust_gamma/")

In [ ]:
for image in imagelist:
    img = Image.open("Art_Dataset/test_img/{}".format(image))
    img = adjust_gamma(img)
    img.save("Art_Dataset/adjust_gamma/{}".format(image))

In [ ]:
#####  fake lighting ###########################

In [25]:
import numpy as np
def fake_light(image, tilesize=30):
    WIDTH, HEIGHT = image.size
    ah = np.random.randint(0,1200)
    aw = np.random.randint(0,1600)
    print("ah:", ah)
    print("aw:", aw)
    for x in range(0, WIDTH, tilesize):
        for y in range(0, HEIGHT, tilesize):
            br = int(255 * (1 - x / float(ah) * y /float(aw)))
            tile = Image.new("RGBA", (tilesize, tilesize), (255,255,255,128))
            image.paste((br,br,br), (x, y, x + tilesize, y + tilesize), mask=tile)
    return image

In [ ]:
imagelist = [file for file in os.listdir("Art_Dataset/test_img/") if file.endswith('.png')]
print(len(imagelist))
if not os.path.exists("Art_Dataset/fake_img"):
    os.mkdir("Art_Dataset/fake_img/")

In [ ]:
for image in imagelist:
    img = Image.open("Art_Dataset/test_img/{}".format(image))
    img = fake_light(img)
    img.save("Art_Dataset/fake_img/{}".format(image))

In [26]:

image = Image.open('{}.png'.format(51))
img = fake_light(image)
img.save('fake.jpg')

ah: 563
aw: 1395


In [ ]:
###########################

In [34]:
from PIL import Image
img = Image.open('spotlight_51.jpg')

In [35]:
with open('gt_51.txt') as f:
    num = 8
    for line in f.read().splitlines():
        cor = line.split(',')
        x1 = int(cor[0])
        y1 = int(cor[1])
        x2 = int(cor[4])
        y2 = int(cor[5])
        image = img.crop((x1,y1,x2,y2))
        image.save('Art_Dataset/store_test_image/{}.ppm'.format(num))
        num=num+1